# Carbon Use Efficiency of Marine Heterotrophs

In [1]:
import cobra
import os
import pandas as pd

In [2]:
import gem2cue.utils
import gem2cue.calculate_cue

## Set-up

In [3]:
# Read in the meta-data file as a data frame
meta_df = pd.read_csv('IAMM_models/strain_metadata.tsv', sep='\t')

In [4]:
# For each row, make a Strain object with the model and metadata from the file
strain_list = []
for index, row in meta_df.iterrows():
    strain_list.append(gem2cue.utils.Strain(row['strain'],
                                            cobra.io.read_sbml_model(os.path.join('IAMM_models', row['model_file'])),
                                            row['gc_content'],
                                            row['genome_length']))

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


## Analysis

### Experiment 0: Unconstrained

In [5]:
# Set the media to be the same as in one of the strains
media = strain_list[0].model.medium

# Make an experiment for each Strain
unconstrained_experiments = [gem2cue.utils.Experiment(strain, media) for strain in strain_list]

In [7]:
strain_list[0].model.medium

{'EX_12dgr160_e': 1000.0,
 'EX_12dgr180_e': 1000.0,
 'EX_1ag160_e': 1000.0,
 'EX_1ag180_e': 1000.0,
 'EX_1ag181d9_e': 1000.0,
 'EX_1ag182d9d12_e': 1000.0,
 'EX_23camp_e': 1000.0,
 'EX_23ccmp_e': 1000.0,
 'EX_23cgmp_e': 1000.0,
 'EX_23cump_e': 1000.0,
 'EX_2m35mdntha_e': 1000.0,
 'EX_35dnta_e': 1000.0,
 'EX_3h4atb_e': 1000.0,
 'EX_3mb_e': 1000.0,
 'EX_4abut_e': 1000.0,
 'EX_4hbz_e': 1000.0,
 'EX_4oxptn_e': 1000.0,
 'EX_6atha_e': 1000.0,
 'EX_LalaLglu_e': 1000.0,
 'EX_abg4_e': 1000.0,
 'EX_ac_e': 1000.0,
 'EX_acald_e': 1000.0,
 'EX_acgam_e': 1000.0,
 'EX_actn__R_e': 1000.0,
 'EX_ad_e': 1000.0,
 'EX_ala__L_e': 1000.0,
 'EX_alahis_e': 1000.0,
 'EX_alaleu_e': 1000.0,
 'EX_alathr_e': 1000.0,
 'EX_alatrp_e': 1000.0,
 'EX_algac_MG_14_e': 1000.0,
 'EX_algac_MG_23_e': 1000.0,
 'EX_algac_MG_32_e': 1000.0,
 'EX_algac_MG_41_e': 1000.0,
 'EX_algac__M_e': 1000.0,
 'EX_arab__D_e': 1000.0,
 'EX_arbt6p_e': 1000.0,
 'EX_arbt_e': 1000.0,
 'EX_arg__L_e': 1000.0,
 'EX_asn__L_e': 1000.0,
 'EX_aso3_e': 1000.0

In [10]:
strain_list[0].model.medium.keys()

dict_keys(['EX_12dgr160_e', 'EX_12dgr180_e', 'EX_1ag160_e', 'EX_1ag180_e', 'EX_1ag181d9_e', 'EX_1ag182d9d12_e', 'EX_23camp_e', 'EX_23ccmp_e', 'EX_23cgmp_e', 'EX_23cump_e', 'EX_2m35mdntha_e', 'EX_35dnta_e', 'EX_3h4atb_e', 'EX_3mb_e', 'EX_4abut_e', 'EX_4hbz_e', 'EX_4oxptn_e', 'EX_6atha_e', 'EX_LalaLglu_e', 'EX_abg4_e', 'EX_ac_e', 'EX_acald_e', 'EX_acgam_e', 'EX_actn__R_e', 'EX_ad_e', 'EX_ala__L_e', 'EX_alahis_e', 'EX_alaleu_e', 'EX_alathr_e', 'EX_alatrp_e', 'EX_algac_MG_14_e', 'EX_algac_MG_23_e', 'EX_algac_MG_32_e', 'EX_algac_MG_41_e', 'EX_algac__M_e', 'EX_arab__D_e', 'EX_arbt6p_e', 'EX_arbt_e', 'EX_arg__L_e', 'EX_asn__L_e', 'EX_aso3_e', 'EX_aso4_e', 'EX_asp__L_e', 'EX_but_e', 'EX_butso3_e', 'EX_bz_e', 'EX_ca2_e', 'EX_chol_e', 'EX_chols_e', 'EX_cinnm_e', 'EX_cl_e', 'EX_cmcbtt_e', 'EX_co2_e', 'EX_cobalt2_e', 'EX_cu2_e', 'EX_cys__L_e', 'EX_cytd_e', 'EX_dag181d9_e', 'EX_dag182d9d12_e', 'EX_dca_e', 'EX_ddca_e', 'EX_drib_e', 'EX_enter_e', 'EX_etoh_e', 'EX_fald_e', 'EX_fcmcbtt_e', 'EX_fe2_e', 

In [6]:
# Calculate CUE for all of the Experiments
for exp in unconstrained_experiments:
    exp.CUE(ex_nomenclature = {'C_e'})

/home/helen/miniconda3/envs/CUE/lib/python3.10/site-packages/cobra/core/dictlist.py:214: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if current_length is 0:
/home/helen/miniconda3/envs/CUE/lib/python3.10/site-packages/cobra/core/dictlist.py:214: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if current_length is 0:


KeyError: 'EX_12dgr160_e'

In [ ]:
# Pull out all of the CUE results
[exp.cue for exp in unconstrained_experiments]

### Experiment 1: Glucose

In [ ]:
# Make the medium I want all to use for all of the 

### Experiment 2: Acetate

## Visualization